# Advanced Git

## Stashing: The Interrupt Handler

You're halfway through `feature/experiment` when the boss says "Fix the login bug now!"

```bash
# Save everything (including untracked files with -u)
git stash push -m "WIP: experimenting with power function" -u

# Working directory is now clean
git status  # "nothing to commit"

# Switch to main, fix bug...
git switch main
# ... fix, commit, push ...

# Return to your experiment
git switch feature/experiment
git stash pop  # Applies changes and removes from stash
```

**Stash commands:**
```bash
git stash list                    # See all stashes
git stash show -p stash@{0}       # Preview stash contents  
git stash apply stash@{0}         # Apply but keep in stash
git stash drop stash@{0}          # Delete specific stash
git stash clear                   # Delete all stashes
```

## The Undo Ladder (From Safe to Dangerous)

| Situation | Command | Scope | Danger |
|-----------|---------|-------|--------|
| Discard uncommitted edits | `git restore <file>` | Working Dir only | Low |
| Unstage a file | `git restore --staged <file>` | Staging Area only | Low |
| Fix last commit (unpushed) | `git commit --amend` | Rewrites last commit | Medium (changes SHA) |
| Undo a pushed commit safely | `git revert <commit>` | Creates new commit | Low (safe for team) |
| Move branch pointer back | `git reset --soft HEAD~1` | History only | Medium |
| Nuclear option | `git reset --hard <commit>` | Working Dir + History | **HIGH** (data loss) |

## Safe Undos (Working Directory & Staging)

**Discard local changes (Working Directory):**
```bash
# Oops, I messed up calculator.py
git restore calculator.py
# File returns to last committed state
```

**Unstage a file (Staging Area):**
```bash
git add wrong_file.py
# Oops, didn't mean to stage that
git restore --staged wrong_file.py
# File is untracked/unstaged again (but changes remain in Working Dir)
```

## Commit Surgery

### Amend (The "Oops" Fix)
You just committed but forgot to add a file or made a typo in the message:

```bash
# Add the forgotten file
git add forgotten_file.py

# Amend the commit (changes the SHA hash)
git commit --amend -m "feat: Add calculator with power function"

# If already pushed to remote, you must force push (DANGER):
git push --force-with-lease
# --force-with-lease is safer than --force - it checks if someone else pushed first
```

**Warning:** Amending changes the commit's SHA. If others pulled your old commit, you've rewritten public history.

### Revert (The Safe Undo)
You committed something bad yesterday, but others already pulled it:

```bash
# Creates a NEW commit that undoes the old one
git revert a1b2c3d

# History becomes:
# A --- B --- C --- D (revert commit)
```

**Why revert?** It preserves the fact that the bad commit existed (for audit trails) while neutralizing its effect.

## Reset: The Time Machine

**Soft Reset (Undo commit, keep changes staged):**
```bash
git reset --soft HEAD~1
# Commit disappears, changes remain in Staging Area
# Useful for: Combining last 2 commits into 1
```

**Mixed Reset (Undo commit, unstage changes):**
```bash
git reset --mixed HEAD~1  
# Commit disappears, changes in Working Directory
# Useful for: Rethinking what goes in the commit
```

**Hard Reset (🔥 Nuclear):**
```bash
git reset --hard HEAD~1
# Commit disappears, changes disappear forever
# Useful for: "Burn it all down" (only when totally sure)
```

## The Reflog: Your Safety Net

Even if you `reset --hard` or delete a branch, Git keeps a log of every HEAD movement for 30 days.

```bash
git reflog

# Output example:
a1b2c3d HEAD@{0}: reset: moving to HEAD~1
b2c3d4e HEAD@{1}: commit: Add power function
c3d4e5f HEAD@{2}: checkout: moving from main to feature
```

**Recovery:**
```bash
# Oops, I needed that commit I just reset away!
git reset --hard HEAD@{1}

# Or recover a deleted branch
git reflog | grep "commit: Add power"
# Note the hash, then:
git checkout -b recovered-branch b2c3d4e
```

**The Reflog is local only.** It saves you from yourself, but not from others' force-pushes.

## Rebase: Rewriting History for Cleanliness

### What is Rebase?
Rebase moves or combines a sequence of commits to a new base commit. It literally "re-bases" your branch onto another point in history.

**Visual Explanation:**

Before rebase:
```
main:       A --- B --- C
                      \
feature:                D --- E --- F
```

After `git rebase main`:
```
main:       A --- B --- C
                         \
feature:                   D' --- E' --- F'
```

Notice: `D`, `E`, `F` become `D'`, `E'`, `F'`—they have new hashes because their parent changed from `C` to `C` (technically same parent, but if main moved forward, they get replayed on top).

### Why Rebase?

**The Problem with Merge:**
```
main:    A --- B --- C --- G (merge commit)
                \         /
feature:          D --- E
```
The merge commit `G` clutters history. It looks like parallel universes collided.

**The Rebase Solution:**
```
main:    A --- B --- C --- D' --- E'
```
Linear history! It looks like you wrote `D` and `E` after `C` was done, even though you started when `B` was latest.

### When to Rebase vs Merge

| Scenario | Use | Result |
|----------|-----|--------|
| Local feature branch, not yet pushed | `rebase main` | Clean linear history |
| Shared branch (origin/main) | `merge` | Preserve true history |
| Integrating finished feature | `merge --no-ff` | Preserve feature existence |

### Interactive Rebase: The History Editor

Clean up messy commits before sharing:

```bash
# Last 3 commits
git rebase -i HEAD~3

# Editor opens with:
pick a1b2c3d Add power function
pick b2c3d4e WIP: fix bug  
pick c3d4e5f Fix typo in comment

# Change to:
pick a1b2c3d Add power function
squash b2c3d4e WIP: fix bug
fixup c3d4e5f Fix typo in comment
```

**Interactive Commands:**
- `pick` (p): Use commit as-is
- `reword` (r): Change commit message
- `edit` (e): Stop to amend the commit (add files, split commit)
- `squash` (s): Combine with previous commit (keeps both messages, you edit combined message)
- `fixup` (f): Combine with previous commit (discards this message)
- `drop` (d): Delete commit entirely
- `exec` (x): Run a shell command after this commit

### The Golden Rule of Rebasing

**Never rebase commits that exist outside your repository!**

If you rebase commits already pushed to `origin/feature`, and someone else pulled that branch, you create alternate reality. When you force-push, their next pull will create a merge nightmare.

**Safe Rebase Workflow:**
```bash
# 1. Update main
git switch main
git pull origin main

# 2. Rebase your local feature onto latest main
git switch feature
git rebase main
# (fix any conflicts during rebase)

# 3. If you already pushed feature (and you're alone on it)
git push --force-with-lease origin feature
# --force-with-lease checks nobody else pushed before overwriting
```

### Rebase vs Merge: Practical Example

**Scenario:** You worked on `feature` for 3 days. Main moved forward with 10 new commits.

**Merge approach:**
```bash
git switch feature
git merge main
# Creates merge commit, keeps exact history of when you branched
# History shows parallel development (truthful but messy)
```

**Rebase approach:**
```bash
git switch feature  
git rebase main
# Replays your commits on top of latest main
# History looks like you started today (clean but rewritten)
# May need to resolve conflicts multiple times (once per commit)
```

**Professional Tip:** Rebase local branches for clean history. Merge shared branches to preserve truth.

## GUI Tools
- [GUI Clients](https://git-scm.com/downloads/guis)
- Extension inside IDEs

## More About Git
- For more information read [Pro Git book](https://git-scm.com/book/en/v2)